## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gmaps_key

# Configure gmaps API key
gmaps.configure(api_key = gmaps_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
new_city_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
new_city_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,78.35,76,3,15.95,clear sky
1,1,Goundi,TD,9.3627,17.3660,70.57,21,99,6.15,overcast clouds
2,2,Shestakovo,RU,56.4915,103.9600,-0.60,95,97,2.95,overcast clouds
3,3,Mandalgovi,MN,45.7625,106.2708,3.85,68,0,5.06,clear sky
4,4,Campana,AR,-34.1687,-58.9591,78.66,96,98,3.71,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = new_city_df.loc[(new_city_df["Max Temp"] <= max_temp) & \
                                       (new_city_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,78.35,76,3,15.95,clear sky
4,4,Campana,AR,-34.1687,-58.9591,78.66,96,98,3.71,light rain
6,6,Maragogi,BR,-9.0122,-35.2225,78.76,79,77,8.88,broken clouds
7,7,Kavieng,PG,-2.5744,150.7967,83.88,75,89,8.25,light rain
10,10,Hambantota,LK,6.1241,81.1185,76.12,76,100,8.93,overcast clouds
11,11,Rikitea,PF,-23.1203,-134.9692,78.19,76,100,16.96,light rain
15,15,Mahebourg,MU,-20.4081,57.7000,78.26,69,75,10.36,broken clouds
17,17,Kathu,TH,7.9178,98.3332,86.63,71,54,10.16,broken clouds
21,21,Mar Del Plata,AR,-38.0023,-57.5575,76.08,100,75,18.01,broken clouds
22,22,Sao Filipe,CV,14.8961,-24.4956,75.72,59,2,10.49,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                165
City                   165
Country                165
Lat                    165
Lng                    165
Max Temp               165
Humidity               165
Cloudiness             165
Wind Speed             165
Current Description    165
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis = 0, how = "any", thresh = None, subset = None, inplace = False)

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.35,clear sky,-9.8000,-139.0333,
4,Campana,AR,78.66,light rain,-34.1687,-58.9591,
6,Maragogi,BR,78.76,broken clouds,-9.0122,-35.2225,
7,Kavieng,PG,83.88,light rain,-2.5744,150.7967,
10,Hambantota,LK,76.12,overcast clouds,6.1241,81.1185,
11,Rikitea,PF,78.19,light rain,-23.1203,-134.9692,
15,Mahebourg,MU,78.26,broken clouds,-20.4081,57.7000,
17,Kathu,TH,86.63,broken clouds,7.9178,98.3332,
21,Mar Del Plata,AR,76.08,broken clouds,-38.0023,-57.5575,
22,Sao Filipe,CV,75.72,clear sky,14.8961,-24.4956,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmaps_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)    
        

Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.35,clear sky,-9.8000,-139.0333,Villa Enata
4,Campana,AR,78.66,light rain,-34.1687,-58.9591,Hotel Morada Suites
6,Maragogi,BR,78.76,broken clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
7,Kavieng,PG,83.88,light rain,-2.5744,150.7967,Nusa Island Retreat
10,Hambantota,LK,76.12,overcast clouds,6.1241,81.1185,Bungalow 63
11,Rikitea,PF,78.19,light rain,-23.1203,-134.9692,People ThankYou
15,Mahebourg,MU,78.26,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
17,Kathu,TH,86.63,broken clouds,7.9178,98.3332,Novotel Phuket Resort
21,Mar Del Plata,AR,76.08,broken clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
22,Sao Filipe,CV,75.72,clear sky,14.8961,-24.4956,Hotel Xaguate


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis = 0, how = "any", thresh = None, subset = None, inplace = False)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.35,clear sky,-9.8000,-139.0333,Villa Enata
4,Campana,AR,78.66,light rain,-34.1687,-58.9591,Hotel Morada Suites
6,Maragogi,BR,78.76,broken clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
7,Kavieng,PG,83.88,light rain,-2.5744,150.7967,Nusa Island Retreat
10,Hambantota,LK,76.12,overcast clouds,6.1241,81.1185,Bungalow 63


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))